# Априорные и апостериорные предикторы востребованности контента

Это исследование, по сути, является просто упражнением на сравнительный анализ различных методов кодирования кактегоральных данных. То, что мы кодируем именно априорный признак, это случайность, продиктованная спецификой конкретных данных. 

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn.linear_model as skl

## Чтение данных и предварительный анализ

In [2]:
df = pd.read_csv('data/data.csv')
df

,Название видео,Тип видео,Клики по элементам конечной заставки,Показы элементов конечной заставки,Показы тизеров,Постоянные зрители,Новые зрители,Среднее число просмотров одним пользователем,Уникальные зрители,Средний процент просмотра (%),...,Новые подписчики,Новые комментарии,Поделились,"Отметки ""Не нравится""","Отметки ""Нравится""",Просмотры,Время просмотра (часы),Подписчики,Показы,CTR для значков видео (%)
0,Линейная зависимость и линейная независимость....,Тема,9.0,792.0,54.0,325.0,1100.0,1.2833,1465.0,23.93,...,8.0,1.0,17.0,4.0,40.0,1880.0,90.9871,7.0,8889.0,14.77
1,Вычисление определителей по методу Гаусса. Тема,Тема,7.0,467.0,56.0,211.0,585.0,1.2736,815.0,28.95,...,6.0,1.0,10.0,2.0,12.0,1038.0,49.8394,6.0,3114.0,15.70
2,Ограниченные последовательности. Тема,Тема,8.0,372.0,27.0,138.0,726.0,1.1998,876.0,21.47,...,3.0,0.0,7.0,0.0,10.0,1051.0,41.9261,3.0,3272.0,18.12
3,Приведение кривой второго порядка к каноническ...,Пример,1.0,121.0,15.0,195.0,511.0,1.3243,740.0,10.13,...,1.0,0.0,8.0,2.0,8.0,980.0,38.4302,1.0,4192.0,13.62
4,Решение неоднородных линейных систем. Тема,Тема,1.0,165.0,24.0,181.0,307.0,1.2854,508.0,25.59,...,0.0,1.0,5.0,0.0,12.0,653.0,37.7780,0.0,2002.0,10.39
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,Сумма и скалярное кратное линейных операторов....,Ответы,0.0,16.0,0.0,9.0,0.0,1.3333,9.0,41.48,...,0.0,0.0,0.0,0.0,1.0,12.0,0.3056,0.0,437.0,1.60
496,Задачи на максимум и минимум. Ответы,Ответы,0.0,4.0,0.0,3.0,5.0,1.2500,8.0,20.91,...,0.0,0.0,2.0,0.0,0.0,10.0,0.3050,0.0,145.0,4.14
497,Непрерывность функции двух переменных. Вопросы,Вопросы,1.0,45.0,2.0,24.0,0.0,1.1250,24.0,38.94,...,0.0,0.0,0.0,0.0,0.0,27.0,0.3038,0.0,251.0,9.16
498,Градиент. Вопросы,Вопросы,0.0,16.0,5.0,20.0,2.0,1.3043,23.0,30.30,...,0.0,0.0,0.0,0.0,0.0,30.0,0.3030,0.0,511.0,3.72


In [3]:
df.describe()

,Клики по элементам конечной заставки,Показы элементов конечной заставки,Показы тизеров,Постоянные зрители,Новые зрители,Среднее число просмотров одним пользователем,Уникальные зрители,Средний процент просмотра (%),Отказались от подписки,Новые подписчики,Новые комментарии,Поделились,"Отметки ""Не нравится""","Отметки ""Нравится""",Просмотры,Время просмотра (часы),Подписчики,Показы,CTR для значков видео (%)
count,500.000000,500.00000,500.000000,500.000000,500.000000,500.000000,500.000000,500.000000,500.000000,500.0000,500.000000,500.000000,500.000000,500.000000,500.000000,500.000000,500.000000,500.000000,500.000000
mean,0.508000,58.78600,3.942000,45.594000,32.586000,1.241784,79.674000,35.109640,0.010000,0.3600,0.056000,0.640000,0.064000,1.486000,100.530000,4.023387,0.350000,621.738000,5.969620
std,1.048874,79.56626,7.925419,49.457591,94.640204,0.171804,133.822347,13.616501,0.099598,1.0002,0.254939,1.811343,0.322664,3.079085,170.646446,7.663491,0.976426,797.378602,3.318488
min,0.000000,0.00000,0.000000,2.000000,0.000000,1.000000,3.000000,9.520000,0.000000,0.0000,0.000000,0.000000,-1.000000,-2.000000,5.000000,0.302600,-1.000000,39.000000,0.000000
25%,0.000000,16.00000,0.000000,14.000000,1.000000,1.138275,20.000000,25.170000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,23.000000,0.586175,0.000000,217.500000,3.700000
50%,0.000000,32.00000,0.000000,26.000000,4.500000,1.221600,34.500000,32.440000,0.000000,0.0000,0.000000,0.000000,0.000000,1.000000,42.500000,1.319000,0.000000,409.500000,5.425000
75%,1.000000,67.25000,5.000000,56.000000,21.000000,1.307700,81.250000,43.862500,0.000000,0.0000,0.000000,0.000000,0.000000,2.000000,99.250000,3.562425,0.000000,663.750000,7.690000
max,9.000000,792.00000,56.000000,338.000000,1100.000000,2.600000,1465.000000,74.570000,1.000000,9.0000,2.000000,17.000000,4.000000,40.000000,1880.000000,90.987100,9.000000,8889.000000,21.240000


In [4]:
df.describe(include = 'object')

,Название видео,Тип видео
count,500,500
unique,500,5
top,Линейная зависимость и линейная независимость....,Тема
freq,1,204


## Построение и обучение регрессионной модели на числовых данных

In [5]:
df.drop(columns = ['Название видео', 'Тип видео'])

,Клики по элементам конечной заставки,Показы элементов конечной заставки,Показы тизеров,Постоянные зрители,Новые зрители,Среднее число просмотров одним пользователем,Уникальные зрители,Средний процент просмотра (%),Отказались от подписки,Новые подписчики,Новые комментарии,Поделились,"Отметки ""Не нравится""","Отметки ""Нравится""",Просмотры,Время просмотра (часы),Подписчики,Показы,CTR для значков видео (%)
0,9.0,792.0,54.0,325.0,1100.0,1.2833,1465.0,23.93,1.0,8.0,1.0,17.0,4.0,40.0,1880.0,90.9871,7.0,8889.0,14.77
1,7.0,467.0,56.0,211.0,585.0,1.2736,815.0,28.95,0.0,6.0,1.0,10.0,2.0,12.0,1038.0,49.8394,6.0,3114.0,15.70
2,8.0,372.0,27.0,138.0,726.0,1.1998,876.0,21.47,0.0,3.0,0.0,7.0,0.0,10.0,1051.0,41.9261,3.0,3272.0,18.12
3,1.0,121.0,15.0,195.0,511.0,1.3243,740.0,10.13,0.0,1.0,0.0,8.0,2.0,8.0,980.0,38.4302,1.0,4192.0,13.62
4,1.0,165.0,24.0,181.0,307.0,1.2854,508.0,25.59,0.0,0.0,1.0,5.0,0.0,12.0,653.0,37.7780,0.0,2002.0,10.39
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,0.0,16.0,0.0,9.0,0.0,1.3333,9.0,41.48,0.0,0.0,0.0,0.0,0.0,1.0,12.0,0.3056,0.0,437.0,1.60
496,0.0,4.0,0.0,3.0,5.0,1.2500,8.0,20.91,0.0,0.0,0.0,2.0,0.0,0.0,10.0,0.3050,0.0,145.0,4.14
497,1.0,45.0,2.0,24.0,0.0,1.1250,24.0,38.94,0.0,0.0,0.0,0.0,0.0,0.0,27.0,0.3038,0.0,251.0,9.16
498,0.0,16.0,5.0,20.0,2.0,1.3043,23.0,30.30,0.0,0.0,0.0,0.0,0.0,0.0,30.0,0.3030,0.0,511.0,3.72


In [6]:
X = df.drop(columns = ['Название видео', 'Тип видео', 'Время просмотра (часы)']).to_numpy()
y = df['Время просмотра (часы)'].to_numpy()

model = skl.LinearRegression()
model.fit(X, y)
model.score(X, y)

0.9134275159364447

## Включение категориальной переменной

### Слепое ранжирование

#### Чтение данных

In [7]:
df = pd.read_csv('data/data.csv')
df = df.drop(columns = ['Название видео'])
df

,Тип видео,Клики по элементам конечной заставки,Показы элементов конечной заставки,Показы тизеров,Постоянные зрители,Новые зрители,Среднее число просмотров одним пользователем,Уникальные зрители,Средний процент просмотра (%),Отказались от подписки,Новые подписчики,Новые комментарии,Поделились,"Отметки ""Не нравится""","Отметки ""Нравится""",Просмотры,Время просмотра (часы),Подписчики,Показы,CTR для значков видео (%)
0,Тема,9.0,792.0,54.0,325.0,1100.0,1.2833,1465.0,23.93,1.0,8.0,1.0,17.0,4.0,40.0,1880.0,90.9871,7.0,8889.0,14.77
1,Тема,7.0,467.0,56.0,211.0,585.0,1.2736,815.0,28.95,0.0,6.0,1.0,10.0,2.0,12.0,1038.0,49.8394,6.0,3114.0,15.70
2,Тема,8.0,372.0,27.0,138.0,726.0,1.1998,876.0,21.47,0.0,3.0,0.0,7.0,0.0,10.0,1051.0,41.9261,3.0,3272.0,18.12
3,Пример,1.0,121.0,15.0,195.0,511.0,1.3243,740.0,10.13,0.0,1.0,0.0,8.0,2.0,8.0,980.0,38.4302,1.0,4192.0,13.62
4,Тема,1.0,165.0,24.0,181.0,307.0,1.2854,508.0,25.59,0.0,0.0,1.0,5.0,0.0,12.0,653.0,37.7780,0.0,2002.0,10.39
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,Ответы,0.0,16.0,0.0,9.0,0.0,1.3333,9.0,41.48,0.0,0.0,0.0,0.0,0.0,1.0,12.0,0.3056,0.0,437.0,1.60
496,Ответы,0.0,4.0,0.0,3.0,5.0,1.2500,8.0,20.91,0.0,0.0,0.0,2.0,0.0,0.0,10.0,0.3050,0.0,145.0,4.14
497,Вопросы,1.0,45.0,2.0,24.0,0.0,1.1250,24.0,38.94,0.0,0.0,0.0,0.0,0.0,0.0,27.0,0.3038,0.0,251.0,9.16
498,Вопросы,0.0,16.0,5.0,20.0,2.0,1.3043,23.0,30.30,0.0,0.0,0.0,0.0,0.0,0.0,30.0,0.3030,0.0,511.0,3.72


#### Вывод уникальных значений предиктора 'Тип видео'

In [8]:
df['Тип видео'].unique()

array(['Тема', 'Пример', 'Ответы', 'Другое', 'Вопросы'], dtype=object)

#### Кодирование методом replace

In [9]:
df = df.replace({'Тип видео':{'Тема': 0, 'Вопросы': 1, 'Ответы': 2, 'Пример': 3, 'Другое': 4}})
df

,Тип видео,Клики по элементам конечной заставки,Показы элементов конечной заставки,Показы тизеров,Постоянные зрители,Новые зрители,Среднее число просмотров одним пользователем,Уникальные зрители,Средний процент просмотра (%),Отказались от подписки,Новые подписчики,Новые комментарии,Поделились,"Отметки ""Не нравится""","Отметки ""Нравится""",Просмотры,Время просмотра (часы),Подписчики,Показы,CTR для значков видео (%)
0,0,9.0,792.0,54.0,325.0,1100.0,1.2833,1465.0,23.93,1.0,8.0,1.0,17.0,4.0,40.0,1880.0,90.9871,7.0,8889.0,14.77
1,0,7.0,467.0,56.0,211.0,585.0,1.2736,815.0,28.95,0.0,6.0,1.0,10.0,2.0,12.0,1038.0,49.8394,6.0,3114.0,15.70
2,0,8.0,372.0,27.0,138.0,726.0,1.1998,876.0,21.47,0.0,3.0,0.0,7.0,0.0,10.0,1051.0,41.9261,3.0,3272.0,18.12
3,3,1.0,121.0,15.0,195.0,511.0,1.3243,740.0,10.13,0.0,1.0,0.0,8.0,2.0,8.0,980.0,38.4302,1.0,4192.0,13.62
4,0,1.0,165.0,24.0,181.0,307.0,1.2854,508.0,25.59,0.0,0.0,1.0,5.0,0.0,12.0,653.0,37.7780,0.0,2002.0,10.39
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,2,0.0,16.0,0.0,9.0,0.0,1.3333,9.0,41.48,0.0,0.0,0.0,0.0,0.0,1.0,12.0,0.3056,0.0,437.0,1.60
496,2,0.0,4.0,0.0,3.0,5.0,1.2500,8.0,20.91,0.0,0.0,0.0,2.0,0.0,0.0,10.0,0.3050,0.0,145.0,4.14
497,1,1.0,45.0,2.0,24.0,0.0,1.1250,24.0,38.94,0.0,0.0,0.0,0.0,0.0,0.0,27.0,0.3038,0.0,251.0,9.16
498,1,0.0,16.0,5.0,20.0,2.0,1.3043,23.0,30.30,0.0,0.0,0.0,0.0,0.0,0.0,30.0,0.3030,0.0,511.0,3.72


#### Построение и обучение модели

In [10]:
X = df.drop(columns = ['Время просмотра (часы)']).to_numpy()
y = df['Время просмотра (часы)'].to_numpy()

model = skl.LinearRegression()
model.fit(X, y)
model.score(X, y)

0.920598151385312

### Ранжирование по среднему значению целевой функции

#### Чтение данных

In [11]:
df = pd.read_csv('data/data.csv')
df = df.drop(columns = ['Название видео'])
df

,Тип видео,Клики по элементам конечной заставки,Показы элементов конечной заставки,Показы тизеров,Постоянные зрители,Новые зрители,Среднее число просмотров одним пользователем,Уникальные зрители,Средний процент просмотра (%),Отказались от подписки,Новые подписчики,Новые комментарии,Поделились,"Отметки ""Не нравится""","Отметки ""Нравится""",Просмотры,Время просмотра (часы),Подписчики,Показы,CTR для значков видео (%)
0,Тема,9.0,792.0,54.0,325.0,1100.0,1.2833,1465.0,23.93,1.0,8.0,1.0,17.0,4.0,40.0,1880.0,90.9871,7.0,8889.0,14.77
1,Тема,7.0,467.0,56.0,211.0,585.0,1.2736,815.0,28.95,0.0,6.0,1.0,10.0,2.0,12.0,1038.0,49.8394,6.0,3114.0,15.70
2,Тема,8.0,372.0,27.0,138.0,726.0,1.1998,876.0,21.47,0.0,3.0,0.0,7.0,0.0,10.0,1051.0,41.9261,3.0,3272.0,18.12
3,Пример,1.0,121.0,15.0,195.0,511.0,1.3243,740.0,10.13,0.0,1.0,0.0,8.0,2.0,8.0,980.0,38.4302,1.0,4192.0,13.62
4,Тема,1.0,165.0,24.0,181.0,307.0,1.2854,508.0,25.59,0.0,0.0,1.0,5.0,0.0,12.0,653.0,37.7780,0.0,2002.0,10.39
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,Ответы,0.0,16.0,0.0,9.0,0.0,1.3333,9.0,41.48,0.0,0.0,0.0,0.0,0.0,1.0,12.0,0.3056,0.0,437.0,1.60
496,Ответы,0.0,4.0,0.0,3.0,5.0,1.2500,8.0,20.91,0.0,0.0,0.0,2.0,0.0,0.0,10.0,0.3050,0.0,145.0,4.14
497,Вопросы,1.0,45.0,2.0,24.0,0.0,1.1250,24.0,38.94,0.0,0.0,0.0,0.0,0.0,0.0,27.0,0.3038,0.0,251.0,9.16
498,Вопросы,0.0,16.0,5.0,20.0,2.0,1.3043,23.0,30.30,0.0,0.0,0.0,0.0,0.0,0.0,30.0,0.3030,0.0,511.0,3.72


#### Ранжирование по среднему значению

In [12]:
print('Тема', df.loc[df['Тип видео'] == 'Тема']['Время просмотра (часы)'].mean())
print('Вопросы', df.loc[df['Тип видео'] == 'Вопросы']['Время просмотра (часы)'].mean())
print('Ответы', df.loc[df['Тип видео'] == 'Ответы']['Время просмотра (часы)'].mean())
print('Пример', df.loc[df['Тип видео'] == 'Пример']['Время просмотра (часы)'].mean())
print('Другое', df.loc[df['Тип видео'] == 'Другое']['Время просмотра (часы)'].mean())

Тема 7.353307352941177
Вопросы 0.7831059523809524
Ответы 1.4482031007751939
Пример 5.612536842105264
Другое 1.0165177777777776


In [13]:
df = df.replace({'Тип видео':{'Тема': 4, 'Вопросы': 0, 'Ответы': 2, 'Пример': 3, 'Другое': 1}})
df

,Тип видео,Клики по элементам конечной заставки,Показы элементов конечной заставки,Показы тизеров,Постоянные зрители,Новые зрители,Среднее число просмотров одним пользователем,Уникальные зрители,Средний процент просмотра (%),Отказались от подписки,Новые подписчики,Новые комментарии,Поделились,"Отметки ""Не нравится""","Отметки ""Нравится""",Просмотры,Время просмотра (часы),Подписчики,Показы,CTR для значков видео (%)
0,4,9.0,792.0,54.0,325.0,1100.0,1.2833,1465.0,23.93,1.0,8.0,1.0,17.0,4.0,40.0,1880.0,90.9871,7.0,8889.0,14.77
1,4,7.0,467.0,56.0,211.0,585.0,1.2736,815.0,28.95,0.0,6.0,1.0,10.0,2.0,12.0,1038.0,49.8394,6.0,3114.0,15.70
2,4,8.0,372.0,27.0,138.0,726.0,1.1998,876.0,21.47,0.0,3.0,0.0,7.0,0.0,10.0,1051.0,41.9261,3.0,3272.0,18.12
3,3,1.0,121.0,15.0,195.0,511.0,1.3243,740.0,10.13,0.0,1.0,0.0,8.0,2.0,8.0,980.0,38.4302,1.0,4192.0,13.62
4,4,1.0,165.0,24.0,181.0,307.0,1.2854,508.0,25.59,0.0,0.0,1.0,5.0,0.0,12.0,653.0,37.7780,0.0,2002.0,10.39
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,2,0.0,16.0,0.0,9.0,0.0,1.3333,9.0,41.48,0.0,0.0,0.0,0.0,0.0,1.0,12.0,0.3056,0.0,437.0,1.60
496,2,0.0,4.0,0.0,3.0,5.0,1.2500,8.0,20.91,0.0,0.0,0.0,2.0,0.0,0.0,10.0,0.3050,0.0,145.0,4.14
497,0,1.0,45.0,2.0,24.0,0.0,1.1250,24.0,38.94,0.0,0.0,0.0,0.0,0.0,0.0,27.0,0.3038,0.0,251.0,9.16
498,0,0.0,16.0,5.0,20.0,2.0,1.3043,23.0,30.30,0.0,0.0,0.0,0.0,0.0,0.0,30.0,0.3030,0.0,511.0,3.72


#### Построение и обучение модели

In [14]:
X = df.drop(columns = ['Время просмотра (часы)']).to_numpy()
y = df['Время просмотра (часы)'].to_numpy()

model = skl.LinearRegression()
model.fit(X, y)
model.score(X, y)

0.9306219403057472

### Дробное ранжирование

#### Чтение данных

In [15]:
df = pd.read_csv('data/data.csv')
df = df.drop(columns = ['Название видео'])
df

,Тип видео,Клики по элементам конечной заставки,Показы элементов конечной заставки,Показы тизеров,Постоянные зрители,Новые зрители,Среднее число просмотров одним пользователем,Уникальные зрители,Средний процент просмотра (%),Отказались от подписки,Новые подписчики,Новые комментарии,Поделились,"Отметки ""Не нравится""","Отметки ""Нравится""",Просмотры,Время просмотра (часы),Подписчики,Показы,CTR для значков видео (%)
0,Тема,9.0,792.0,54.0,325.0,1100.0,1.2833,1465.0,23.93,1.0,8.0,1.0,17.0,4.0,40.0,1880.0,90.9871,7.0,8889.0,14.77
1,Тема,7.0,467.0,56.0,211.0,585.0,1.2736,815.0,28.95,0.0,6.0,1.0,10.0,2.0,12.0,1038.0,49.8394,6.0,3114.0,15.70
2,Тема,8.0,372.0,27.0,138.0,726.0,1.1998,876.0,21.47,0.0,3.0,0.0,7.0,0.0,10.0,1051.0,41.9261,3.0,3272.0,18.12
3,Пример,1.0,121.0,15.0,195.0,511.0,1.3243,740.0,10.13,0.0,1.0,0.0,8.0,2.0,8.0,980.0,38.4302,1.0,4192.0,13.62
4,Тема,1.0,165.0,24.0,181.0,307.0,1.2854,508.0,25.59,0.0,0.0,1.0,5.0,0.0,12.0,653.0,37.7780,0.0,2002.0,10.39
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,Ответы,0.0,16.0,0.0,9.0,0.0,1.3333,9.0,41.48,0.0,0.0,0.0,0.0,0.0,1.0,12.0,0.3056,0.0,437.0,1.60
496,Ответы,0.0,4.0,0.0,3.0,5.0,1.2500,8.0,20.91,0.0,0.0,0.0,2.0,0.0,0.0,10.0,0.3050,0.0,145.0,4.14
497,Вопросы,1.0,45.0,2.0,24.0,0.0,1.1250,24.0,38.94,0.0,0.0,0.0,0.0,0.0,0.0,27.0,0.3038,0.0,251.0,9.16
498,Вопросы,0.0,16.0,5.0,20.0,2.0,1.3043,23.0,30.30,0.0,0.0,0.0,0.0,0.0,0.0,30.0,0.3030,0.0,511.0,3.72


#### Замена строковых значений методом replace

In [16]:
df = df.replace({'Тип видео':{'Тема': df.loc[df['Тип видео'] == 'Тема']['Время просмотра (часы)'].mean(), 
                              'Вопросы': df.loc[df['Тип видео'] == 'Вопросы']['Время просмотра (часы)'].mean(), 
                              'Ответы': df.loc[df['Тип видео'] == 'Ответы']['Время просмотра (часы)'].mean(), 
                              'Пример': df.loc[df['Тип видео'] == 'Пример']['Время просмотра (часы)'].mean(),
                              'Другое': df.loc[df['Тип видео'] == 'Другое']['Время просмотра (часы)'].mean()}})
df

,Тип видео,Клики по элементам конечной заставки,Показы элементов конечной заставки,Показы тизеров,Постоянные зрители,Новые зрители,Среднее число просмотров одним пользователем,Уникальные зрители,Средний процент просмотра (%),Отказались от подписки,Новые подписчики,Новые комментарии,Поделились,"Отметки ""Не нравится""","Отметки ""Нравится""",Просмотры,Время просмотра (часы),Подписчики,Показы,CTR для значков видео (%)
0,7.353307,9.0,792.0,54.0,325.0,1100.0,1.2833,1465.0,23.93,1.0,8.0,1.0,17.0,4.0,40.0,1880.0,90.9871,7.0,8889.0,14.77
1,7.353307,7.0,467.0,56.0,211.0,585.0,1.2736,815.0,28.95,0.0,6.0,1.0,10.0,2.0,12.0,1038.0,49.8394,6.0,3114.0,15.70
2,7.353307,8.0,372.0,27.0,138.0,726.0,1.1998,876.0,21.47,0.0,3.0,0.0,7.0,0.0,10.0,1051.0,41.9261,3.0,3272.0,18.12
3,5.612537,1.0,121.0,15.0,195.0,511.0,1.3243,740.0,10.13,0.0,1.0,0.0,8.0,2.0,8.0,980.0,38.4302,1.0,4192.0,13.62
4,7.353307,1.0,165.0,24.0,181.0,307.0,1.2854,508.0,25.59,0.0,0.0,1.0,5.0,0.0,12.0,653.0,37.7780,0.0,2002.0,10.39
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,1.448203,0.0,16.0,0.0,9.0,0.0,1.3333,9.0,41.48,0.0,0.0,0.0,0.0,0.0,1.0,12.0,0.3056,0.0,437.0,1.60
496,1.448203,0.0,4.0,0.0,3.0,5.0,1.2500,8.0,20.91,0.0,0.0,0.0,2.0,0.0,0.0,10.0,0.3050,0.0,145.0,4.14
497,0.783106,1.0,45.0,2.0,24.0,0.0,1.1250,24.0,38.94,0.0,0.0,0.0,0.0,0.0,0.0,27.0,0.3038,0.0,251.0,9.16
498,0.783106,0.0,16.0,5.0,20.0,2.0,1.3043,23.0,30.30,0.0,0.0,0.0,0.0,0.0,0.0,30.0,0.3030,0.0,511.0,3.72


#### Построение и обучение модели

In [17]:
X = df.drop(columns = ['Время просмотра (часы)']).to_numpy()
y = df['Время просмотра (часы)'].to_numpy()

model = skl.LinearRegression()
model.fit(X, y)
model.score(X, y)

0.9297648014452768

### Кодирование методом OneHotEncoding

#### Чтение данных

In [18]:
df = pd.read_csv('data/data.csv')
df = df.drop(columns = ['Название видео'])
df

,Тип видео,Клики по элементам конечной заставки,Показы элементов конечной заставки,Показы тизеров,Постоянные зрители,Новые зрители,Среднее число просмотров одним пользователем,Уникальные зрители,Средний процент просмотра (%),Отказались от подписки,Новые подписчики,Новые комментарии,Поделились,"Отметки ""Не нравится""","Отметки ""Нравится""",Просмотры,Время просмотра (часы),Подписчики,Показы,CTR для значков видео (%)
0,Тема,9.0,792.0,54.0,325.0,1100.0,1.2833,1465.0,23.93,1.0,8.0,1.0,17.0,4.0,40.0,1880.0,90.9871,7.0,8889.0,14.77
1,Тема,7.0,467.0,56.0,211.0,585.0,1.2736,815.0,28.95,0.0,6.0,1.0,10.0,2.0,12.0,1038.0,49.8394,6.0,3114.0,15.70
2,Тема,8.0,372.0,27.0,138.0,726.0,1.1998,876.0,21.47,0.0,3.0,0.0,7.0,0.0,10.0,1051.0,41.9261,3.0,3272.0,18.12
3,Пример,1.0,121.0,15.0,195.0,511.0,1.3243,740.0,10.13,0.0,1.0,0.0,8.0,2.0,8.0,980.0,38.4302,1.0,4192.0,13.62
4,Тема,1.0,165.0,24.0,181.0,307.0,1.2854,508.0,25.59,0.0,0.0,1.0,5.0,0.0,12.0,653.0,37.7780,0.0,2002.0,10.39
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,Ответы,0.0,16.0,0.0,9.0,0.0,1.3333,9.0,41.48,0.0,0.0,0.0,0.0,0.0,1.0,12.0,0.3056,0.0,437.0,1.60
496,Ответы,0.0,4.0,0.0,3.0,5.0,1.2500,8.0,20.91,0.0,0.0,0.0,2.0,0.0,0.0,10.0,0.3050,0.0,145.0,4.14
497,Вопросы,1.0,45.0,2.0,24.0,0.0,1.1250,24.0,38.94,0.0,0.0,0.0,0.0,0.0,0.0,27.0,0.3038,0.0,251.0,9.16
498,Вопросы,0.0,16.0,5.0,20.0,2.0,1.3043,23.0,30.30,0.0,0.0,0.0,0.0,0.0,0.0,30.0,0.3030,0.0,511.0,3.72


#### Кодирование

In [19]:
df = pd.get_dummies(df, columns = ['Тип видео'], dtype = 'int') 
df

,Клики по элементам конечной заставки,Показы элементов конечной заставки,Показы тизеров,Постоянные зрители,Новые зрители,Среднее число просмотров одним пользователем,Уникальные зрители,Средний процент просмотра (%),Отказались от подписки,Новые подписчики,...,Просмотры,Время просмотра (часы),Подписчики,Показы,CTR для значков видео (%),Тип видео_Вопросы,Тип видео_Другое,Тип видео_Ответы,Тип видео_Пример,Тип видео_Тема
0,9.0,792.0,54.0,325.0,1100.0,1.2833,1465.0,23.93,1.0,8.0,...,1880.0,90.9871,7.0,8889.0,14.77,0,0,0,0,1
1,7.0,467.0,56.0,211.0,585.0,1.2736,815.0,28.95,0.0,6.0,...,1038.0,49.8394,6.0,3114.0,15.70,0,0,0,0,1
2,8.0,372.0,27.0,138.0,726.0,1.1998,876.0,21.47,0.0,3.0,...,1051.0,41.9261,3.0,3272.0,18.12,0,0,0,0,1
3,1.0,121.0,15.0,195.0,511.0,1.3243,740.0,10.13,0.0,1.0,...,980.0,38.4302,1.0,4192.0,13.62,0,0,0,1,0
4,1.0,165.0,24.0,181.0,307.0,1.2854,508.0,25.59,0.0,0.0,...,653.0,37.7780,0.0,2002.0,10.39,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,0.0,16.0,0.0,9.0,0.0,1.3333,9.0,41.48,0.0,0.0,...,12.0,0.3056,0.0,437.0,1.60,0,0,1,0,0
496,0.0,4.0,0.0,3.0,5.0,1.2500,8.0,20.91,0.0,0.0,...,10.0,0.3050,0.0,145.0,4.14,0,0,1,0,0
497,1.0,45.0,2.0,24.0,0.0,1.1250,24.0,38.94,0.0,0.0,...,27.0,0.3038,0.0,251.0,9.16,1,0,0,0,0
498,0.0,16.0,5.0,20.0,2.0,1.3043,23.0,30.30,0.0,0.0,...,30.0,0.3030,0.0,511.0,3.72,1,0,0,0,0


#### Построение и обучение модели

In [20]:
X = df.drop(columns = ['Время просмотра (часы)']).to_numpy()
y = df['Время просмотра (часы)'].to_numpy()

model = skl.LinearRegression()
model.fit(X, y)
model.score(X, y)

0.9345395820674294